In [41]:
import numpy as np
import pandas as pd
import sys
from timeit import default_timer as timer
import matplotlib.pyplot as plt
%matplotlib inline

sys.path.append("/Users/hakunahahannah/Documents/Projects/Spiking/Analysis/SpikeAnalysisToolbox/")

import spikeAnalysisToolsV2.data_loading as data
import spikeAnalysisToolsV2.firing_rates as firing
import spikeAnalysisToolsV2.helper as helper
import spikeAnalysisToolsV2.overviews as overview
import spikeAnalysisToolsV2.combine_stimuli as combine
import spikeAnalysisToolsV2.plotting as spikeplot
import spikeAnalysisToolsV2.information_scores as info

In [42]:
## set the Masterpath to the folder where your output is saved
n_epochs = 400

masterpath = "/Users/hakunahahannah/Documents/Projects/Spiking/Build/output"
## set the subfolder to the Simulation you want to analyse 

subfolders = [
    "20Ex20IdNetworkTEST_400_epochs"
            ]
## if more than the inital epoch is needed *1 needs to be run
extensions =  ["initial"] # ["testing/epoch123"]
object_list = data.load_testing_stimuli_info(masterpath+"/" + subfolders[0]) #assuming all the subfolders have the same
n_stimuli = np.sum(obj['count'] for obj in object_list)


# info_neurons is just an array of the information from above. This makes it easier to run the functions and pass the information. 
# info_times same for times
network_architecture = dict(
    num_exc_neurons_per_layer = 64*64,
    num_inh_neurons_per_layer = 32*32,
    num_layers = 4,
    #total_per_layer = num_exc_neurons_per_layer + num_inh_neurons_per_layer,
    #total_network = total_per_layer * num_layers,
    #num_stimuli = 16
)

info_times = dict(
    length_of_stimulus = 2.0,
    num_stimuli = n_stimuli,
    time_start = 1.5,
    time_end = 1.9
)

#objects_in_training = [
#    object_list[0]['indices'] + object_list[1]['indices'] + object_list[2]['indices'] + object_list[3]['indices'],
#    object_list[4]['indices'] + object_list[5]['indices'] + object_list[6]['indices'] + object_list[7]['indices'],
#]
# These Objects were bound together in training with temporal trace. so it should have learned information about them. 
# print(objects_in_training)
# object_indices = objects_in_training
stimuli_names = data.load_testing_stimuli_names(masterpath+"/" + subfolders[0])
print(stimuli_names)
object_indices = data.load_testing_stimuli_indices_from_wildcarts(masterpath+"/" + subfolders[0], ["**d*", "**c*"])
print(object_indices)

['obj0:face_1_Happy', 'obj0:face_2_Happy', 'obj0:face_3_Happy', 'obj0:face_4_Happy', 'obj0:face_5_Happy', 'obj0:face_6_Happy', 'obj0:face_7_Happy', 'obj0:face_8_Happy', 'obj0:face_9_Happy', 'obj0:face_10_Happy', 'obj0:face_11_Happy', 'obj0:face_12_Happy', 'obj0:face_13_Happy', 'obj0:face_14_Happy', 'obj0:face_15_Happy', 'obj0:face_16_Happy', 'obj0:face_17_Happy', 'obj0:face_18_Happy', 'obj0:face_19_Happy', 'obj0:face_20_Happy', 'obj1:face_1_Sad', 'obj1:face_2_Sad', 'obj1:face_3_Sad', 'obj1:face_4_Sad', 'obj1:face_5_Sad', 'obj1:face_6_Sad', 'obj1:face_7_Sad', 'obj1:face_8_Sad', 'obj1:face_9_Sad', 'obj1:face_10_Sad', 'obj1:face_11_Sad', 'obj1:face_12_Sad', 'obj1:face_13_Sad', 'obj1:face_14_Sad', 'obj1:face_15_Sad', 'obj1:face_16_Sad', 'obj1:face_17_Sad', 'obj1:face_18_Sad', 'obj1:face_19_Sad', 'obj1:face_20_Sad']
[{'filter': '**d*', 'elements': set(), 'indices': [], 'count': 0}, {'filter': '**c*', 'elements': {'face_13_Happy', 'face_4_Happy', 'face_5_Sad', 'face_11_Sad', 'face_20_Sad', '

In [47]:
final_epoch_rates = rates_subfolders[0][0]

In [48]:
exc_rates, inh_rates = helper.nested_list_of_stimuli_2_np(final_epoch_rates)
exc_rates.shape

(40, 4, 4096)

In [57]:
spikes = data.load_spikes_from_subfolders(masterpath, subfolders, extensions, False)

rates_subfolders = firing.calculate_rates_subfolder(
    spikes,
    network_architecture, 
    info_times)

overview.print_firing_rates_tables(rates_subfolders, info_times, subfolders, extensions, True)

Start
20Ex20IdNetworkTEST_400_epochs
Extension:  initial
Layer 0 
              Exc Mean  Exc Min  Exc Max  Inh Mean  Inh Min  Inh Max
Stimulus 0   1.786499      0.0     72.5  2.583008      0.0     37.5
Stimulus 1   1.946411      0.0     65.0  2.480469      0.0     42.5
Stimulus 2   1.886597      0.0     97.5  2.221680      0.0     45.0
Stimulus 3   1.984863      0.0     67.5  2.229004      0.0     37.5
Stimulus 4   2.079468      0.0     92.5  2.312012      0.0     35.0
Stimulus 5   1.933594      0.0     85.0  2.377930      0.0     37.5
Stimulus 6   1.892700      0.0     52.5  2.680664      0.0     30.0
Stimulus 7   2.163086      0.0     65.0  2.529297      0.0     37.5
Stimulus 8   1.837769      0.0     65.0  2.343750      0.0     47.5
Stimulus 9   1.792603      0.0     82.5  2.631836      0.0     37.5
Stimulus 10  1.869507      0.0     85.0  2.211914      0.0     35.0
Stimulus 11  2.044678      0.0     82.5  1.950684      0.0     47.5
Stimulus 12  1.700439      0.0     52.5  2.741699

## Minimal Response to prefered stimulus

In [80]:
exh_min_objects_before, inh_min_objects_before = info.min_response_to_one_transform(rates_subfolders[0][0], object_indices)

print("first Epoch - Minimal response to the sensitive stimulus")
spikeplot.show_activity_in_layers(exh_min_objects_before, inh_min_objects_before, cmap='bwr')

[{'filter': '**d*', 'elements': set(), 'indices': [], 'count': 0}, {'filter': '**c*', 'elements': {'face_13_Happy', 'face_4_Happy', 'face_5_Sad', 'face_11_Sad', 'face_20_Sad', 'face_13_Sad', 'face_12_Sad', 'face_2_Sad', 'face_6_Sad', 'face_17_Sad', 'face_9_Happy', 'face_16_Happy', 'face_18_Happy', 'face_12_Happy', 'face_9_Sad', 'face_7_Happy', 'face_1_Sad', 'face_14_Happy', 'face_4_Sad', 'face_10_Sad', 'face_3_Sad', 'face_18_Sad', 'face_7_Sad', 'face_17_Happy', 'face_1_Happy', 'face_8_Happy', 'face_15_Sad', 'face_6_Happy', 'face_14_Sad', 'face_5_Happy', 'face_19_Sad', 'face_10_Happy', 'face_3_Happy', 'face_19_Happy', 'face_2_Happy', 'face_20_Happy', 'face_15_Happy', 'face_11_Happy', 'face_8_Sad', 'face_16_Sad'}, 'indices': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39], 'count': 40}]


/Users/hakunahahannah/Documents/Projects/Spiking/Analysis/SpikeAnalysisToolbox/spikeAnalysisToolsV2/helper.py:307: RuntimeWarning: invalid value encountered in true_divide
  transformed = (data - mean) / sigma


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

## Single Cell Information

In [7]:
exh_info_before, inh_info_before = info.firing_rates_to_single_cell_information(rates_subfolders[0][0], object_indices, n_bins=3, calc_inhibitory=True)
exh_info_before.max()
print("first Epoch - Information")
spikeplot.show_activity_in_layers(exh_info_before, inh_info_before, cmap='bwr')

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
spikeplot.plot_ranked_neurons(exh_info_before, "Information for test with CT ", 500, ["object 0", "object 1"])

In [96]:
exh_info_after, inh_info_after = info.firing_rates_to_single_cell_information(rates_subfolders[0][1], object_indices, n_bins=3, calc_inhibitory=True)
print("last Epoch - Information")
spikeplot.plot_ranked_neurons(exh_info_after, "Information for test with unseen ", 500, ["object 0", "object 1"])
spikeplot.show_activity_in_layers(exh_info_after, inh_info_after, cmap='bwr')


IndexError: list index out of range

In [ ]:
spikeplot.plot_information_measure_advancement(exh_info_before, exh_info_after)

## Average response to prefered Object (as multiple of std)

In [ ]:
exh_avg_higher_response, inh_avg_higher_response = info.average_higher_z_response_to_object(rates_subfolders[0][0], object_indices, 0)
print(exh_info_before.max())
print("first Epoch - Information")
spikeplot.show_activity_in_layers(exh_avg_higher_response, inh_avg_higher_response, cmap='bwr')

In [ ]:
print(exh_avg_higher_response.shape)
spikeplot.plot_ranked_neurons(exh_avg_higher_response, "Higher response for object: ", 100, ["object 0", "object 1"])

# Probability of NOT drawing these samples from the same distribution

In [ ]:
exc_t_p, inh_t_p = info.t_test_p_value(rates_subfolders[0][0], object_indices)
spikeplot.show_activity_in_layers(exc_t_p, inh_t_p, cmap='bwr')

In [ ]:
print(exc_t_p.max())
spikeplot.plot_ranked_neurons(exc_t_p, "Probability of not beeing from the same distribution: ", 1000, ["1-p"], vmin=0.9999999999999999)

# Mutual Information

In [ ]:
exc_mutual, inh_mutual = info.firing_rates_to_mutual_information(rates_subfolders[0][0], object_indices, n_bins=3, calc_inhibitory=True)
spikeplot.show_activity_in_layers(exc_mutual, inh_mutual, cmap='bwr')

In [ ]:
spikeplot.plot_ranked_neurons(inh_mutual, "Top neurons ranked for mutual information", 100, ["mutual information"])